In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

In [2]:
%matplotlib inline

In [3]:
import time
import math

In [4]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

# `np.sum` 

https://realpython.com/numpy-array-programming/

In [ ]:
array = np.array([[[[2, -3], [3, 2]], [[-4, -1], [-5, 1]], [[-7, -5], [-1, 6]],
                   [[-5, 0], [-4, 2]]],
                  [[[-1, 4], [6, 1]], [[-2, -3], [-5, 5]], [[-2, -8], [-1, 7]],
                   [[-1, 8], [-4, 2]]]])
array.shape
np.sum(array, axis=3)

https://stackoverflow.com/questions/57830433/how-to-get-the-euclid-distance-from-specified-array-in-numpy#57830445

In [ ]:
def find_eucledian_distances(a_points, b_points):
    return np.sqrt(np.sum((a_points - b_points)**2, axis=-1))


a = np.array([[3.0, 4.0], [-3.0, -6.0], [-2.5, 6.3]])
b = np.array([[0.0, 0.0], [2.0, 6.0], [4.5, -8.3]])
(a - b) ** 2
np.sqrt(np.sum((a - b)**2, axis=-1))

<span style="font-family: New York Times; font-size: 1em; color: green;">
    
you should `np.sum` along the last axis. if you don't specific the axis=-1, the `np.sum()` will sum all of the elements.

# arithmetic between np.array

https://stackoverflow.com/questions/57806304/numpy-elementwise-multiplication-with-multiple-columns#57806420

In [ ]:
a = np.array([[1, 2, 3], [4, 5, 6]])
b = np.array([[7, 8, 9, 10], [11, 12, 13, 14]])
c = np.zeros((2, 4, 3))

for i, row in enumerate(b):
    for j, col in enumerate(row):
        print('Mult:', a[i, :], '*', col)
        c[i, j, :] = a[i, :] * col

print(c[0, :, :])
print(c[1, :, :])

In [ ]:
a = np.array([[1, 2, 3], [4, 5, 6]])
b = np.array([[7, 8, 9, 10], [11, 12, 13, 14]])
print([item * k for item in a for k in b.ravel()])

In [ ]:
a = np.array([[1, 2, 3], [4, 5, 6]])
b = np.array([[7, 8, 9, 10], [11, 12, 13, 14]])
a[:,None,:] * b[:,:,None]

https://stackoverflow.com/questions/57843027/add-small-np-array-to-larger-np-array#57843046

In [ ]:
import numpy as np
a = np.arange(1, 11)
b = np.arange(1,3)
print('a:',a)
print('b:',b)
a[0::2] += b[0]
a[1::2] += b[1]
print('a:',a)

In [ ]:
a = np.arange(1, 11)
b = np.arange(1,3)
a.reshape(-1, len(b)) 
a.reshape(-1, len(b)) + b 
(a.reshape(-1, len(b)) + b).ravel()

# `np.reshape`

https://docs.scipy.org/doc/numpy/reference/generated/numpy.reshape.html#numpy.reshape

In [ ]:
a = np.array([[1,2,3, 4], [5,6,7,8]])
print(np.reshape(a, (-2)) )
print("___________________________________")
print(np.reshape(a, (2, 2,-2)) )
print("___________________________________")
print(np.reshape(a, (2, -1,4)) )

# `np.array.tolist`

In [ ]:
arr = np.array([[1, 1, 1, 1], [2, 2, 2, 2], [3, 3, 3, 3], [4, 4, 4, 4]])
lens = [1, 2, 3, 4]
[a[:i].tolist() for a, i in zip(arr, lens)]

In [ ]:
num1 = [2, 2, 0]
num2 = [1, 0]
list(str( int(''.join(map(str, num1))) * int(''.join(map(str, num2)))))

In [ ]:
def multiply(num1, num2):
    if num1 == [0] or num2 == [0]:
        return [0]
    sign = -1 if (num1[0] < 0) ^ (num2[0] < 0) else 1
    num1[0] = abs(num1[0])
    num2[0] = abs(num2[0])
    res = [0] * (len(num1) + len(num2) + 1) # space O(n + m)
    for i in range(len(num1) - 1, -1, -1):
        for j in range(len(num2) - 1, -1, -1):
            res[i + j + 1] += num1[i] * num2[j]
            res[i + j] += res[i + j + 1] // 10
            res[i + j + 1] %= 10
    res[0] *= sign

    #res = res[next((i for i, x in enumerate(res) if x != 0), len(res)):] or [0]
    return res

multiply(num1, num2)

# Large array 

https://stackoverflow.com/questions/57565758/efficient-way-to-write-a-list-to-file-in-python/57565815#57565815

In [ ]:
import numpy as np
mylist = np.arange(1,n+1)
list_to_write = func(mylist)
np.savetxt('test.out', x, delimiter=',') 

# `np.mean`

https://stackoverflow.com/questions/53178018/average-of-elements-in-a-subarray

https://stackoverflow.com/questions/57814566/get-average-of-the-numpy-ndarray

In [ ]:
m = np.array([[[0, 1, 2], [3, 4, 5], [6, 7, 8]],
              [[9, 10, 11], [12, 13, 14], [15, 16, 17]],
              [[18, 19, 20], [21, 22, 23], [24, 25, 26]]])

In [ ]:
m.shape
np.mean(m, axis=0)
np.mean(m, axis=(0,1))
np.mean(m, axis=2)
m[0]

In [ ]:
[np.mean(m[i]) for i in range(m.shape[0])]
list(map(np.mean, m))
np.mean(m, (1, 2))

# `np.where`

In [ ]:
m = np.array([[[0, 1, 2], [3, 4, 5], [6, 7, 8]],
              [[9, 10, 11], [12, 13, 14], [15, 16, 17]],
              [[18, 19, 20], [21, 22, 23], [24, 25, 26]]])

In [ ]:
np.where(m == np.amax(m)) 

In [ ]:
a = np.array([1,2,3,4])
x, = np.where(a==a.max())
print(x)

In [ ]:
data = [[0.4, 1.5, 2.6],
        [3.4, 0.2, 0.0],
        [np.nan, 3.2, 1.0],
        [1.0, 4.6, np.nan]]

In [ ]:
np.where(np.isnan(data))

In [ ]:
x = np.array(data)
np.isnan(x) 

In [ ]:
import warnings
warnings.filterwarnings('ignore')
data = [[0.4, 1.5, 2.6],
        [3.4, 0.2, 0.0],
        [np.nan, 3.2, 1.0],
        [1.0, 4.6, np.nan]]
x = np.array(data) 
i, j = np.where(x > 1 )
warnings.filterwarnings('ignore')
for a, b in zip(i, j):
    print('lon: {} lat: {} value: {}'.format(a, b, x[a,b]))

In [ ]:
array= np.array([[1, 2, 3],[np.nan, np.nan, np.nan], [3, 4, 5]])
np.nan_to_num(array)

In [ ]:
A = np.array([[ 0.57383254,  0.10132767,  0.86211639,  0.35402222],
       [ 0.20238346,  0.93204519,  0.84563318,  0.68373515],
       [ 0.5795946 ,  0.24307856,  0.56676058,  0.08502582],
       [ 0.27188428,  0.0630682 ,  0.9762359 ,  0.50456657],
       [ 0.6522969 ,  0.85018875,  0.22728716,  0.82851854]]) 
target = np.array([ 0.57959463 ,  0.24307856,  0.56676058,  0.08502582])

In [ ]:
np.where(A==target)[0]

In [ ]:
result, = np.where(np.all(A == target, axis=1))
print(result)

https://stackoverflow.com/questions/57833856/how-to-convert-ax-y-z-to-x0-xn-y0-yn-z0-zn

In [ ]:
A = np.ones((1,2))
np.where(A)[0]
np.where(A)[1]
np.ravel(A)

# `np.rotate`

In [ ]:
x = np.arange(25).reshape(5, -1) 
rotate(x, angle=45, reshape=False)

In [ ]:
def switch1(s):
    s = list(s)
    for i in range(len(s)):
        if s[i] == 'a':
            s[i] = 'b'
        if s[i] == 'b':
            s[i] = 'a'
    return s

def switch2(s):
    s = list(s)
    for i in range(len(s)):
        if s[i] == 'a':
            s[i] = 'b'
        elif s[i] == 'b':
            s[i] = 'a'
    return s

switch1('abab')
switch2('abab')

# `np.searchsorted`

* https://stackoverflow.com/questions/57121825/vectorized-approach-to-extract-the-rows-of-the-end-point-array-that-contains-the

In [ ]:
np.random.seed(0)

size = 32000
base_arr = np.arange(size)*10
base_arr
t1 = np.random.randint(0,6, size)+base_arr
t2 = np.random.randint(5,10, size)+base_arr

A = np.vstack((t1,t2)).T
V = np.sort(np.random.randint(0,10,3*size)+np.repeat(base_arr,3))

In [ ]:
A[np.searchsorted(v,A[:,0],'left')!=np.searchsorted(v,A[:,1],'right')]

In [ ]:
def find_bounding_intervals(A, v):
    rows = []
    i = 0
    for row in A:
        while all(v[i] < row):
            i += 1
        if row[0] <= v[i] <= row[1]:
            rows.append(row)
    return np.array(rows)
find_bounding_intervals(A, V)

# `np.consum`

In [ ]:
x = np.array([[np.nan, np.nan, np.nan, np.nan, np.nan],
       [np.nan, np.nan, np.nan, np.nan, np.nan],
       [ 0., np.nan, np.nan, np.nan, np.nan],
       [ 0., np.nan,  0., np.nan,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [np.nan,  0.,  0.,  0.,  0.]])
x.T
np.isnan(x)
np.nonzero(x)
np.where(np.isnan(x))

x[np.nonzero(x)] = 2 
x 

In [ ]:
x = np.array([[5.72756493, 0.        , 0.        ],
          [2.86378246, 4.96021673, 0.        ],
          [5.72756493, 3.30681115, 2.33826859]])

https://stackoverflow.com/questions/57966043/replace-zeros-in-an-array-with-a-continuous-sequence-of-integers


In [ ]:
arr = np.array([[np.nan, np.nan, np.nan, np.nan, np.nan],
       [np.nan, np.nan, np.nan, np.nan, np.nan],
       [ 0., np.nan, np.nan, np.nan, np.nan],
       [ 0., np.nan,  0., np.nan,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [np.nan,  0.,  0.,  0.,  0.]])

mask = ~np.isnan(arr)
np.nan_to_num(arr + 1).ravel('F')
np.nan_to_num(arr + 1).ravel('F').cumsum()
# Applying ravel function in F-style ordering

arr[mask] = np.nan_to_num(arr + 1).ravel('F').cumsum().reshape(arr.shape, order='F')[mask]
arr 

# generate a random array 

https://stackoverflow.com/questions/57814566/get-average-of-the-numpy-ndarray/57814708#57814708

In [14]:
np.random.randint(1,10)
np.random.random((3, 3))
np.random.random_sample((2,2))

3

array([[0.94090967, 0.69684863, 0.24142066],
       [0.79673834, 0.75326164, 0.48490091],
       [0.63579227, 0.48572782, 0.05323919]])

array([[0.67543962, 0.01132504],
       [0.99036466, 0.88944142]])

In [24]:
ShapeOfX = (2,1)
X = np.random.randint(1,10,(ShapeOfX))
X 

array([[6],
       [8]])

In [28]:
A = np.random.randint(1, 10, [2,3])